In [1]:
import datetime as dt
import requests
import random
import time
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from datadiff import diff

import warnings

# Routine Update for METS Online

In [2]:
# URL to the data extraction site
url = "https://metsonline.dosm.gov.my/tradev2/product-coderesult"

# Obtain csrf_token from the website
# csrf_token = 'dG5PZ09oay0nPiIMATBTeTsZHFAgKSJdRQQVKwRYJBQADRckHStGYA=='
csrf_token = 'S09USVY1cWQOYhAbDgMYIS4JAh0FWRdRJzhtcQdyPSAPGyEuOQcECQ=='

# Create headers for all posts
headers = {
"Content-Type": "application/x-www-form-urlencoded; charset=UTF-8", 
"Cookie": "_ga=GA1.1.1701463660.1689306590; mp_f55a9b6a137ab21675ef8724f7864bde_mixpanel=%7B%22distinct_id%22%3A%20%221895284e7dd9de-0ca9b33bffbf56-7e565470-1fa400-1895284e7de4d4%22%2C%22%24device_id%22%3A%20%221895284e7dd9de-0ca9b33bffbf56-7e565470-1fa400-1895284e7de4d4%22%2C%22%24initial_referrer%22%3A%20%22https%3A%2F%2Fwww.dosm.gov.my%2F%22%2C%22%24initial_referring_domain%22%3A%20%22www.dosm.gov.my%22%2C%22%24search_engine%22%3A%20%22google%22%2C%22__timers%22%3A%20%7B%7D%7D; _ga_TF34V4E8EM=GS1.1.1691399745.5.0.1691399745.0.0.0; _ga_F8QG3XBL63=GS1.1.1692449791.1.1.1692449822.0.0.0; _ga_7Q47WMYPD5=GS1.1.1692449791.1.1.1692449838.0.0.0; _ga_HK905W412V=GS1.1.1692449791.1.1.1692449838.0.0.0; PHPSESSID=qp6iqscr24r83sg45qf7n4e6m0; _csrf=569bb962a7369341748d6e9dce0797d77214c3a32c3fcb544fae435ee5d0e154a%3A2%3A%7Bi%3A0%3Bs%3A5%3A%22_csrf%22%3Bi%3A1%3Bs%3A32%3A%22E-DRX6iEeFVTSlf5lw98QGLDDTugo2um%22%3B%7D; BNES_PHPSESSID=wvzRyezmpkBIysicU6hxNIV7+orbJUx9j8u9CVGOobirld8xr1fxvpHo1jFrRP4RG9QER+lj3e5rfauP7VqVK1gxK7J1/mphgInZsvTEa7A=; BNES__csrf=l+AjFbmhLRkh7iLvkrZRCFkBfvnrVpH7gQ5i7bspBDxjm/GvapzsIxEUow8RHXtih4XoM88Ar7lxM/7jxkGJKwy8MHhN+iTsPo9Z0D2WY3pdgs0NBX3kbR7cL0KVFmczOimPMnwhq1Kq8z6YXhUMB1o+rq8oSMBnsV1b+a8jsvAu3AW+SNnqsFpWnTB/ibAfNoEFlUylPuCMpSCoVMl+M2R5oVlUkvBA1ny0W89+AGfxXdYKoUs4i02aeZY7+QrR5b4PkoLHP1CkzAmMpPTuoSjXJJnaYOgI"
}

In [3]:
# To obtain the current datetime
current_year = dt.date.today().year
targeted_year = []
targeted_year.append(current_year)
targeted_year

[2023]

In [4]:
# To start extract data from url via request
data_raw = []

max_retries = 2

for year in targeted_year:
    payload_month = {
    '_csrf': csrf_token,
    'Tradev2[typeofsearch]': 'classification',
    'Tradev2[typedigit]': 7,
    'Tradev2[rangecode1]': 0,
    'Tradev2[rangecode2]': 9,
    # 'Tradev2[code_idcode]': ,
    # 'Tradev2[code_idcodedigit9]': ,
    # 'Tradev2[tradeflow]': ,
    'Tradev2[tradeflow][]': 'exports',
    # 'Tradev2[timeframe]': ,
    'Tradev2[timeframe]': 'month',
    # 'Tradev2[rangeyear]': ,
    # 'Tradev2[rangeyear2]': ,
    # 'Tradev2[rangeyearone]': ,
    # 'Tradev2[rangemonthone]': ,
    'Tradev2[mothdata]': year,
    'Tradev2[mothdata2]': year,
    # 'Tradev2[classification_serch]': ,
    # 'Tradev2[country2]': ,
    'Tradev2[geogroup]': 1,
    'Tradev2[geogroup]': 29,
    'Tradev2[codeshowby]': 'code'
    }
    
    for retry in range(max_retries):
        retry_delay = random.randint(10,15)  # seconds
        try:
            # Process the response here
            monthly_raw = requests.post(url, data=payload_month, headers=headers)
            data_raw.append(monthly_raw) # Store the extracted raw data into a list
            print(f"Routine Data Extraction for year {year} is Successful.")
            time.sleep(5)
            break # Break the retry loop if successful
            
        except (requests.ConnectionError) as e:
            print(f"Error: {e}. Retrying in {retry_delay} seconds...")
            time.sleep(retry_delay)
            
        except Exception as e:
            print(f"An error occurred: {e}")
            break  # Stop retrying if a different error occurs
     
print(data_raw)

Routine Data Extraction for year 2023 is Successful.
[<Response [200]>]


In [5]:
# Create an empty list for storing the raw text from data extraction
data_extracted = []

# To extract only the table value
for raw in data_raw:
    result = BeautifulSoup(raw.text, 'html.parser') # Parse the HTML
    table = result.find('table', class_='table-bordered') # Look up for the table
    
    # Extract table rows
    rows = table.find_all('tr')
    for row in rows:
        cols = row.find_all(['th', 'td'])
        cols = [col.get_text(strip=True) for col in cols]
        data_extracted.append(cols)

In [6]:
# To loop through the sublist in the data_extracted, check for empty sublist, if a sublsit is found empty
# Do not append into the filtered_list
filtered_list = [sublist for sublist in data_extracted if sublist]

division_length = 12
# Using a for loop and iterates over a range of indices starting from 0 and an increamental value = division_length
divided_list = [filtered_list[i:i + division_length] for i in range(0, len(filtered_list), division_length)]

In [7]:
# Create an empty dataframe
data = pd.DataFrame()

for item in divided_list:
    df = pd.DataFrame(item)
    df = df.drop(list(range(0,2)), axis =1)
    df = pd.DataFrame.transpose(df) # Tranpose the dataframe
    last_column_index = df.columns[-1] # Remove the last column
    # Rename the columns name
    df = df.drop(columns=last_column_index)
    columns_label = df.iloc[0] 
    columns_label = columns_label.to_list()
    columns_label[0] = 'YEAR'
    df = df.reset_index(drop=True)
    df.columns = columns_label
    df = df.drop(index=[0], axis=0)
    
    # To concatanete dataframes into one
    data = pd.concat([data, df], ignore_index=True)

In [8]:
# Data Cleaning Part
# Remove rows that contain the total value of a year
clean_data = data[data['YEAR'].apply(lambda x: '-' not in x)]

column_names = clean_data.columns.to_list()
del column_names[0]

# Remove the " , " from the number and convert it as integer.
for col in column_names:
    clean_data[col] = clean_data[col].astype(str).str.replace(',','').astype(np.int64)
    
display(clean_data)
print(clean_data.dtypes)

C:\Users\Albert\AppData\Local\Temp\ipykernel_19572\4226581759.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_data[col] = clean_data[col].astype(str).str.replace(',','').astype(np.int64)
C:\Users\Albert\AppData\Local\Temp\ipykernel_19572\4226581759.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_data[col] = clean_data[col].astype(str).str.replace(',','').astype(np.int64)
C:\Users\Albert\AppData\Local\Temp\ipykernel_19572\4226581759.py:10: SettingWithCopyWarning: 
A value is trying to b

,YEAR,FOOD,BEVERAGES AND TOBACCO,"CRUDE MATERIALS, INEDIBLE","MINERAL FUELS, LUBRICANTS, ETC.",ANIMAL AND VEGETABLE OILS AND FATS,CHEMICALS,MANUFACTURED GOODS,MACHINERY & TRANSPORT EQUIPMENT,MISCELLANEOUS MANUFACTURED ARTICLES,MISCELLANEOUS TRANSACTIONS AND COMMODITIES
0,Jan 2023,3079999429,238257518,2399620561,20908748621,5597732713,8476075452,8705481844,52272605677,10412921901,563816343
1,Feb 2023,3539279631,222258336,2346324739,20323875331,5629973235,9036890095,10081396919,49850907614,10783053535,454585466
2,Mar 2023,4044164918,326561786,2715999243,21429738762,7647033259,9950967381,11464714044,57828441926,13401993529,858633228
3,Apr 2023,3251410911,229753325,1946644340,16984271124,5828048670,7583766017,10194652518,49009132271,9527059374,638171651
4,May 2023,3898340823,270752156,2454013437,21965803909,5767124163,9214928157,10424167008,52998110557,11859102111,657960595
5,Jun 2023,3853964611,241920709,2628562391,18254748708,5696471751,8328581144,11573138564,60365867502,12249123233,759357513
6,Jul 2023,3847711463,246541492,2750910622,15690991412,5840106478,8660909643,10736008753,56197583255,12242805660,538042867


YEAR                                          object
FOOD                                           int64
BEVERAGES AND TOBACCO                          int64
CRUDE MATERIALS, INEDIBLE                      int64
MINERAL FUELS, LUBRICANTS, ETC.                int64
ANIMAL AND VEGETABLE OILS AND FATS             int64
CHEMICALS                                      int64
MANUFACTURED GOODS                             int64
MACHINERY & TRANSPORT EQUIPMENT                int64
MISCELLANEOUS MANUFACTURED ARTICLES            int64
MISCELLANEOUS TRANSACTIONS AND COMMODITIES     int64
dtype: object


In [9]:
file_path = ".\export-data-malaysia.csv"
old_df = pd.read_csv(file_path)

In [10]:
# To filter out the year for checking
filter_year = str(targeted_year[0])
filtered_df = old_df[old_df['YEAR'].apply(lambda x : filter_year in x)].reset_index(drop=True)
filtered_df

,YEAR,FOOD,BEVERAGES AND TOBACCO,"CRUDE MATERIALS, INEDIBLE","MINERAL FUELS, LUBRICANTS, ETC.",ANIMAL AND VEGETABLE OILS AND FATS,CHEMICALS,MANUFACTURED GOODS,MACHINERY & TRANSPORT EQUIPMENT,MISCELLANEOUS MANUFACTURED ARTICLES,MISCELLANEOUS TRANSACTIONS AND COMMODITIES
0,Jan 2023,3079999429,238257518,2399620561,20908748621,5597732713,8476075452,8705481844,52272605677,10412921901,563816343
1,Feb 2023,3539279631,222258336,2346324739,20323875331,5629973235,9036890095,10081396919,49850907614,10783053535,454585466
2,Mar 2023,4044164918,326561786,2715999243,21429738762,7647033259,9950967381,11464714044,57828441926,13401993529,858633228
3,Apr 2023,3251410911,229753325,1946644340,16984271124,5828048670,7583766017,10194652518,49009132271,9527059374,638171651
4,May 2023,3898340823,270752156,2454013437,21965803909,5767124163,9214928157,10424167008,52998110557,11859102111,657960595
5,Jun 2023,3853964611,241920709,2628562391,18254748708,5696471751,8328581144,11573138564,60365867502,12249123233,759357513
6,Jul 2023,3847711463,246541492,2750910622,15690991412,5840106478,8660909643,10736008753,56197583255,12242805660,538042867


In [11]:
# To locate the difference between current and previous data
diff_data = clean_data.merge(filtered_df, indicator=True, how='left').loc[lambda x : x['_merge']!='both']
diff_data = diff_data.drop('_merge', axis = 1)
diff_data

,YEAR,FOOD,BEVERAGES AND TOBACCO,"CRUDE MATERIALS, INEDIBLE","MINERAL FUELS, LUBRICANTS, ETC.",ANIMAL AND VEGETABLE OILS AND FATS,CHEMICALS,MANUFACTURED GOODS,MACHINERY & TRANSPORT EQUIPMENT,MISCELLANEOUS MANUFACTURED ARTICLES,MISCELLANEOUS TRANSACTIONS AND COMMODITIES


In [12]:
if diff_data.empty:
    # Concatenation of dataframe is not required if there is no differences in old and current data
    print("No recent update.") 
else:
    # Concatenate the old and current data if differences are found
    updated_data = pd.concat([old_df, diff_data], ignore_index=True)
    print("Update Successful!")

No recent update.


# Routine Update for WBG